In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nelgiriyewithana/emotions")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/emotions


In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import keras

In [ ]:
os.listdir(path)

['text.csv']

In [ ]:
df=pd.read_csv(path+"/text.csv")

In [ ]:
df.head()

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4


In [ ]:
df.drop("Unnamed: 0",axis=1,inplace=True)

In [ ]:
df.head()

,text,label
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4


In [ ]:
x = df["text"].iloc[:10000]
y = df["label"].iloc[:10000]

In [ ]:
y.value_counts(normalize=True)

,proportion
label,
1,0.3396
0,0.2935
3,0.1367
4,0.1149
2,0.0789
5,0.0364


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
def prep(x):
  stop_words=set(stopwords.words("english"))
  x=x.lower()
  x=nlp(x)
  x=[token.lemma_ for token in x if token.text not in stop_words]
  x=" ".join(x)
  return x

In [ ]:
new_x=x.apply(lambda x:prep(x))

In [ ]:
x[1]

'ive enjoyed being able to slouch about relax and unwind and frankly needed it after those last few weeks around the end of uni and the expo i have lately started to find myself feeling a bit listless which is never really a good thing'

In [ ]:
new_x[1]

'enjoy able slouch relax unwind frankly need last week around end uni expo lately start find feel bit listless never really good thing'

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
t=Tokenizer()
t.fit_on_texts(new_x)
seq=t.texts_to_sequences(new_x)

In [ ]:
new_x.map(lambda x:len(x.split())).max()

31

In [ ]:
padd=pad_sequences(seq,maxlen=100)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(padd,y,test_size=0.2,random_state=42,stratify=y)

In [ ]:
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)

class_weights = dict(enumerate(class_weights))

In [ ]:
model = keras.Sequential([
    keras.layers.Embedding(
        input_dim=len(t.word_index)+1,
        output_dim=100,
        input_length=50
    ),
    keras.layers.GRU(512, return_sequences=False),
    keras.layers.Dense(16, activation="leaky_relu", kernel_regularizer=keras.regularizers.L2(0.001)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(8, activation="leaky_relu"),
    # keras.layers.Dropout(0.5),
    keras.layers.Dense(6, activation="softmax")
])

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)


In [ ]:
model.fit(x_train,y_train,epochs=20,validation_split=0.15,class_weight=class_weights,
          batch_size=32,callbacks=[keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

Epoch 1/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.2951 - loss: 1.7916 - val_accuracy: 0.6058 - val_loss: 1.2446
Epoch 2/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.5998 - loss: 1.1909 - val_accuracy: 0.5475 - val_loss: 0.8641
Epoch 3/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.7056 - loss: 0.7241 - val_accuracy: 0.7883 - val_loss: 0.5935
Epoch 4/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.8505 - loss: 0.4698 - val_accuracy: 0.8800 - val_loss: 0.4858
Epoch 5/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.8944 - loss: 0.3487 - val_accuracy: 0.8833 - val_loss: 0.4417
Epoch 6/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9160 - loss: 0.3177 - val_accuracy: 0.8783 - val_loss: 0.4830
Epoch 7/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9361 - loss: 0.2544 - val_accuracy: 0.8850 - val_loss: 0.4361
Epoch 8/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9342 - loss: 0.2435 - val_accu